In [6]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
launch_sites = spacex_df['Launch Site'].unique()
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_sites]

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                    options=dropdown_options,
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0 Kg',
                                                       1000: '1,000 Kg', 
                                                       2000: '2,000 Kg',
                                                       3000: '3,000 Kg',
                                                       4000: '4,000 Kg',
                                                       5000: '5,000 Kg',
                                                       6000: '6,000 Kg',
                                                       7000: '7,000 Kg',
                                                       8000: '8,000 Kg',
                                                       9000: '9,000 Kg',
                                                       10000: '10,000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Filter data to show only successful launches (class == 1)
        successful_launches = spacex_df[spacex_df['class'] == 1]
        
        # Count the successful launches per site
        site_success_counts = successful_launches['Launch Site'].value_counts().reset_index()
        site_success_counts.columns = ['Launch Site', 'Successful Launches']
        
        # Create a pie chart for successful launches across all sites with different colors
        fig = px.pie(site_success_counts, 
                     names='Launch Site', 
                     values='Successful Launches', 
                     title='Successful Launches Across All Sites', 
                     color='Launch Site')  # Color by site
                    
        
    else:
        # If a specific site is selected, show success vs. failure for that site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, 
                     names='class', 
                     title=f'Success vs Failure for {entered_site}', 
                     labels={'class': 'Launch Success'})
    return fig
   
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')])
def update_scatter_plot(selected_site, payload_range):
    # Filter the dataframe based on payload range
    min_payload, max_payload = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) &
                             (spacex_df['Payload Mass (kg)'] <= max_payload)]
    # If a specific site is selected, filter the dataframe for that site
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    # Create a scatter plot using Plotly Express
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',  # Color by booster version
        title=f"Payload Mass vs Launch Outcome for {selected_site if selected_site != 'ALL' else 'All Sites'}",
        labels={'class': 'Launch Outcome', 'Payload Mass (kg)': 'Payload Mass (kg)'},
        hover_data=['Launch Site', 'Booster Version Category']  # Optional: show additional info on hover
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()